In [2]:
!pip install sounddevice

     ------------------------------------ 199.7/199.7 kB 865.2 kB/s eta 0:00:00


In [17]:
import tensorflow_hub as hub
import pandas as pd
from datetime import datetime
import numpy as np
import requests
import zipfile, tarfile
import librosa
import tensorflow as tf
import os, gc, io, time, traceback
from scipy.signal import lfilter, lfilter_zi, firwin
import pywt
import sounddevice as sd
import queue
import threading
from ipywidgets import Button, Layout, ButtonStyle
from IPython.display import display

In [12]:
# Скачиваем модель Yamnet (делаем один раз при первом запуске скрипта)

# URL для скачивания модели
model_url = 'https://storage.googleapis.com/tfhub-modules/google/yamnet/1.tar.gz'
# Путь для сохранения модели
model_path = './yamnet_model'

# Создание каталога, если он не существует
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
# Скачивание и распаковка модели
response = requests.get(model_url)
tar_path = os.path.join(model_path, 'yamnet.tar.gz')
with open(tar_path, 'wb') as file:
    file.write(response.content)

# Распаковка вложенного TAR.GZ архива
with tarfile.open(tar_path, 'r:gz') as tar_ref:
    tar_ref.extractall(model_path)

# Удаление архива после распаковки
os.remove(tar_path)

In [2]:
# загружаем файл классов Yamnet
class_map_path = 'yamnet_class_map_translated.csv'
class_map = pd.read_csv(class_map_path, sep=';', encoding='Windows-1251')

# Загрузка модели
model_path = './yamnet_model'
yamnet_model = hub.load(model_path)

In [4]:
class_map

,index,mid,display_name,display_name_ru
0,0,/m/09x0r,Speech,Речь
1,1,/m/0ytgt,"Child speech, kid speaking","Детская речь, говорящий ребенок"
2,2,/m/01h8n0,Conversation,Разговор
3,3,/m/02qldy,"Narration, monologue","Повествование, монолог"
4,4,/m/0261r1,Babbling,Бормочущий
...,...,...,...,...
516,516,/m/07p_0gm,Throbbing,Пульсирующий
517,517,/m/01jwx6,Vibration,Вибрация
518,518,/m/07c52,Television,Телевидение
519,519,/m/06bz3,Radio,Радио


In [61]:
# Параметры

sample_rate=44100 # частота дискретизации
preemphasis_coef=None #0.25 # Коэффициент преэмфазного шумоподавления
adaptive_filtered=False # Применять ли адаптивную фильтрацию сигнала
wavelet_filtered=False # Применять ли вейвлет-шумоподавление сигнала
n=512 # количество коэффициентов адаптивного фильтра
noise_level = 0.5 # порог адаптивного шумоподавления
probability = 0.25 # порога уверенности классификации моделью YAMNet
time_threshold=5.0 # интервал ожидания повторения такого же события (в секундах)
duration = 5 # размер буферного окна захвата потока
min_event_duration = 0.1 # фильтр коротких импульсных срабатываний

# Cписок интересующих событий (замените значения на те, которые вам нужны)
custom_event_list = [
    "Речь",
    "Животное",
#    "Тишина",
    "Взрыв",
    "Удар",
    "Музыка",
    "Музыкальный инструмент",
    "Кричащий",
    "Кричать",
    "Кричать",
    "Плачущий, рыдающий",
    "Стучащий",
    "Хрипеть",
    "Задыхающийся",
    "Кашель",
    "Чихать",
    "Домашние животные, питомцы"
]

processing = True  # Глобальная переменная для контроля состояния обработки

# Функция адаптивного фильтра для уменьшения шума
def adaptive_filter(y, noise_level, n):
    b = firwin(n, cutoff=noise_level, window='hamming')
    z = lfilter_zi(b, 1)
    filtered_signal, _ = lfilter(b, 1, y, zi=z*y[0])
    return filtered_signal

# Функция шумоподавления на вейвлет-преобразованиях
def wavelet_denoise(data, wavelet='db8', level=3):
    coeffs = pywt.wavedec(data, wavelet, level=level)
    limit = np.std(coeffs[-level]) / 10
    thresholded_coeffs = [pywt.threshold(c, limit, mode='soft') if i == level else c for i, c in enumerate(coeffs)]
    return pywt.waverec(thresholded_coeffs, wavelet)

# Функция для предобработки и анализа потоковых аудиоданных
def preprocess_stream_audio(audio_data, model, sample_rate, preemphasis_coef, adaptive_filtered, wavelet_filtered, class_map, custom_event_list, probability):
    global start_time, processed_time
    # Обработка аудиоданных, поступающих в виде потока
    y = np.array(audio_data).flatten()

    # Применяем преэмфазное шумоподавление для выделения высокочастотных компонентов аудиосигнала
    if preemphasis_coef is not None:
        y = librosa.effects.preemphasis(y, coef=preemphasis_coef)
        #y = librosa.util.normalize(y)

    # Применение адаптивного фильтра для очистки изменяющегося фонового шума
    if adaptive_filtered:
        y = adaptive_filter(y, noise_level, n)
        #y = librosa.util.normalize(y)

    # Применяем вейвлет-шумоподавление для финального удаления шума с сохранением деталей сигнала
    if wavelet_filtered:
        y = wavelet_denoise(y)
        #y = librosa.util.normalize(y)

    # Масштабирование (стандартизация) данных для вытягивания тихих звуков
    #y = (y - np.mean(y)) / np.std(y)
    #y = y / np.max(np.abs(y))
    
    # Анализ с помощью YAMNet
    scores, _, _ = model(y)

    # Определение событий из пользовательского списка с учетом порога вероятности
    events_by_interval = []
    # Проходим по каждому временному шагу в YAMNet-выводе
    for i, score in enumerate(scores):
        interval_events = []
        for class_id, class_probability in enumerate(score):
            if class_map.iloc[class_id, 3] in custom_event_list and class_probability > probability:
                interval_events.append({
                    'description': class_map.iloc[class_id, 3],
                    'probability': class_probability
                })
        
        if interval_events:
            events_by_interval.append({
                'start_time': start_time + processed_time + (i * duration / len(scores)),
                'end_time': start_time + processed_time + ((i + 1) * duration / len(scores)),
                'events': interval_events
            })
    return events_by_interval

# Функция группировки череды одинаковых событий в одно общее событие
def group_events(event_intervals, time_threshold, ongoing_events=None, min_event_duration=min_event_duration):
    if ongoing_events is None:
        ongoing_events = {}

    completed_events = []

    for interval in event_intervals:
        for event in interval['events']:
            event_key = event['description']

            # Если событие уже является продолжающимся
            if event_key in ongoing_events:
                ongoing_event = ongoing_events[event_key]
                if interval['start_time'] - ongoing_event['end_time'] <= time_threshold:
                    # Обновляем время окончания и усредняем вероятность
                    total_duration = ongoing_event['end_time'] - ongoing_event['start_time']
                    new_duration = interval['end_time'] - interval['start_time']
                    ongoing_event['probability'] = (ongoing_event['probability'] * total_duration + event['probability'] * new_duration) / (total_duration + new_duration)
                    ongoing_event['end_time'] = interval['end_time']
                    continue
                else:
                    # Если событие завершилось, добавляем его в completed_events
                    completed_events.append(ongoing_event)
                    del ongoing_events[event_key]

            # Создаем новое событие
            new_event = {
                'description': event_key,
                'probability': event['probability'],
                'start_time': interval['start_time'],
                'end_time': interval['end_time']
            }
            ongoing_events[event_key] = new_event

    # Добавляем в завершенные события те, которые больше не являются активными
    current_time = start_time + processed_time
    for event_key, ongoing_event in list(ongoing_events.items()):
        if ongoing_event['end_time'] < current_time - time_threshold:
            completed_events.append(ongoing_event)
            del ongoing_events[event_key]

    return completed_events, ongoing_events

# Функция для захвата и обработки потоковых аудиоданных
def stream_audio_processing(model, class_map, duration, sample_rate, probability, time_threshold, custom_event_list, min_event_duration=0.5):
    global processing, start_time, processed_time
    start_time = time.time()  # Начальное время обработки
    formatted_start_time = datetime.fromtimestamp(start_time).strftime('%H:%M:%S')
    processed_time = 0    
    cnt = 0
    events_data = []  # Список для сохранения данных о событиях
    audio_data = queue.Queue()
    ongoing_events = {}  # Словарь для продолжающихся событий
    print(f'Начало обработки аудиопотока в {formatted_start_time}\n')
    
    # Callback функция для захвата аудио
    def callback(indata, frames, time, status):
        if status:
            print(status)
        audio_data.put(indata.copy())

    try:
        # Запуск захвата аудио
        with sd.InputStream(callback=callback, channels=1, samplerate=sample_rate, blocksize=int(sample_rate * duration)):
            while processing:
                if not audio_data.empty():
                    data = audio_data.get()
                    event_intervals = preprocess_stream_audio(data, model, sample_rate, preemphasis_coef, adaptive_filtered, wavelet_filtered, class_map, custom_event_list, probability)
                    completed_events, ongoing_events = group_events(event_intervals, time_threshold, ongoing_events, min_event_duration)

                    # Вывод и сохранение завершенных событий
                    for event in completed_events:
                        formatted_start = datetime.fromtimestamp(event["start_time"]).strftime('%H:%M:%S')
                        formatted_end = datetime.fromtimestamp(event["end_time"]).strftime('%H:%M:%S')
                        interval = round((event["end_time"] - event["start_time"]), 2)
                        probability = round(float(event["probability"]), 2)

                        cnt += 1
                        print(f'{cnt}. {event["description"]} (Вероятность: {probability:.2f}), {formatted_start} - {formatted_end}, Длительность: {interval} секунд')
                        events_data.append([cnt, event["description"], probability, formatted_start, formatted_end, interval])

                    processed_time += duration

    except KeyboardInterrupt:
        print('Прервано пользователем')
    except Exception as e:
        print(f'Произошла ошибка: {e}')
        traceback.print_exc()

    finally:
        # Определение времени завершения обработки
        last_time = datetime.fromtimestamp(time.time()).strftime('%H:%M:%S')

        # Обработка последних продолжающихся событий
        for desc, event in ongoing_events.items():
            formatted_start = datetime.fromtimestamp(event["start_time"]).strftime('%H:%M:%S')
            formatted_end = datetime.fromtimestamp(event["end_time"]).strftime('%H:%M:%S')
            interval = round((event["end_time"] - event["start_time"]), 2)
            probability = round(float(event["probability"]), 2)
            cnt += 1
            print(f'{cnt}. {event["description"]} (Вероятность: {probability:.2f}), {formatted_start} - {formatted_end}, Длительность: {interval} секунд')
            events_data.append([cnt, event["description"], probability, formatted_start, formatted_end, interval])

        # Сохранение всех событий в CSV
        if events_data:
            events_df = pd.DataFrame(events_data, columns=['Номер', 'Событие', 'Вероятность', 'Начало', 'Окончание', 'Длительность'])
            directory = os.getcwd()
            csv_filename = f'{int(start_time)}_events.csv'
            csv_path = os.path.join(directory, csv_filename)
            events_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f'\nПотоковая обработка остановлена в {last_time}. События сохранены в {csv_filename}')
        else:
            print(f'Потоковая обработка остановлена в {last_time}. За время наблюдения событий не произошло')

        # Очистка очереди
        while not audio_data.empty():
            audio_data.get()

# Функция для запуска потока обработки аудио
def start_audio_processing_thread(model, class_map, duration, sample_rate, probability, time_threshold, custom_event_list):
    global processing
    processing = True

    def audio_processing_thread():
        stream_audio_processing(model, class_map, duration, sample_rate, probability, time_threshold, custom_event_list)

    processing_thread = threading.Thread(target=audio_processing_thread)
    processing_thread.start()
    
# Функция для остановки потоковой обработки            
def stop_processing(button):
    global processing
    processing = False
    button.description = 'Обработка остановлена. Для возобнавления перезапустите ячейку.'

# Создаем кнопку и событие для остановки обработки
stop_button = Button(
    description='Остановить обработку', 
    layout=Layout(width='500px', height='50px', border='solid'),
    style=ButtonStyle(button_color='lightblue', font_weight='bold')
)
stop_button.on_click(stop_processing)
display(stop_button)

# Запуск обработки потокового аудио
start_audio_processing_thread(model=yamnet_model, class_map=class_map, duration=duration, sample_rate=sample_rate, probability=probability, time_threshold=time_threshold, custom_event_list=custom_event_list)

Button(description='Остановить обработку', layout=Layout(border='solid', height='50px', width='500px'), style=…

Начало обработки аудиопотока в 00:19:40

1. Животное (Вероятность: 0.42), 00:19:46 - 00:19:50, Длительность: 3.39 секунд
2. Музыка (Вероятность: 0.51), 00:19:48 - 00:19:48, Длительность: 0.18 секунд
3. Домашние животные, питомцы (Вероятность: 0.26), 00:19:49 - 00:19:50, Длительность: 0.18 секунд
4. Речь (Вероятность: 0.58), 00:19:47 - 00:19:51, Длительность: 4.82 секунд
5. Речь (Вероятность: 0.76), 00:20:14 - 00:20:14, Длительность: 0.71 секунд
6. Речь (Вероятность: 0.94), 00:20:23 - 00:20:27, Длительность: 4.29 секунд
7. Речь (Вероятность: 0.85), 00:20:34 - 00:20:37, Длительность: 2.86 секунд

Потоковая обработка остановлена в 00:21:25. События сохранены в 1706458780_events.csv
